# Web Scraping Precipitation Data
Scraping [California Department of Water Resources](https://cdec.water.ca.gov/reportapp/javareports?name=PRECIPSUM.201812) for montly precipitation data

In [190]:
import pandas as pd
import numpy as np
import time
import requests
from bs4 import BeautifulSoup

In [191]:
page = requests.get('https://cdec.water.ca.gov/reportapp/javareports?name=PRECIPSUM.201812')

In [192]:
page.status_code

200

In [193]:
soup = BeautifulSoup(page.content, 'html.parser')
rain = soup.find_all(id='PRECIPSUM')
# rain = rain.find
# .find_all(class_='data')

In [194]:
region = rain[1]
# print(first.prettify())
region = region.find_all('tr')[3:13]
region

[<tr><td>NORTH COAST</td><td>0.27</td><td>5</td><td>5</td><td>5</td><td>20</td><td>8</td><td>8</td><td>66%</td><td>69%</td><td>27%</td></tr>,
 <tr><td>SAN FRANCISCO BAY</td><td>0.03</td><td>2</td><td>2</td><td>2</td><td>6</td><td>3</td><td>3</td><td>52.3%</td><td>76%</td><td>27%</td></tr>,
 <tr><td>CENTRAL COAST</td><td>0.06</td><td>3</td><td>3</td><td>3</td><td>10</td><td>3</td><td>3</td><td>50.3%</td><td>87%</td><td>28%</td></tr>,
 <tr><td>SOUTH COAST</td><td>0.06</td><td>3</td><td>3</td><td>3</td><td>14</td><td>12</td><td>12</td><td>97.2%</td><td>103%</td><td>31%</td></tr>,
 <tr><td>SACRAMENTO RIVER</td><td>0.26</td><td>5</td><td>5</td><td>5</td><td>41</td><td>29</td><td>29</td><td>61.9%</td><td>76%</td><td>26%</td></tr>,
 <tr><td>SAN JOAQUIN RIVER</td><td>0.12</td><td>6</td><td>6</td><td>6</td><td>25</td><td>12</td><td>11</td><td>58.9%</td><td>97%</td><td>32%</td></tr>,
 <tr><td>TULARE LAKE</td><td>0.07</td><td>5</td><td>5</td><td>5</td><td>29</td><td>14</td><td>13</td><td>72.6%</t

In [195]:
for i in range(len(region)):
    print(region[i])

<tr><td>NORTH COAST</td><td>0.27</td><td>5</td><td>5</td><td>5</td><td>20</td><td>8</td><td>8</td><td>66%</td><td>69%</td><td>27%</td></tr>
<tr><td>SAN FRANCISCO BAY</td><td>0.03</td><td>2</td><td>2</td><td>2</td><td>6</td><td>3</td><td>3</td><td>52.3%</td><td>76%</td><td>27%</td></tr>
<tr><td>CENTRAL COAST</td><td>0.06</td><td>3</td><td>3</td><td>3</td><td>10</td><td>3</td><td>3</td><td>50.3%</td><td>87%</td><td>28%</td></tr>
<tr><td>SOUTH COAST</td><td>0.06</td><td>3</td><td>3</td><td>3</td><td>14</td><td>12</td><td>12</td><td>97.2%</td><td>103%</td><td>31%</td></tr>
<tr><td>SACRAMENTO RIVER</td><td>0.26</td><td>5</td><td>5</td><td>5</td><td>41</td><td>29</td><td>29</td><td>61.9%</td><td>76%</td><td>26%</td></tr>
<tr><td>SAN JOAQUIN RIVER</td><td>0.12</td><td>6</td><td>6</td><td>6</td><td>25</td><td>12</td><td>11</td><td>58.9%</td><td>97%</td><td>32%</td></tr>
<tr><td>TULARE LAKE</td><td>0.07</td><td>5</td><td>5</td><td>5</td><td>29</td><td>14</td><td>13</td><td>72.6%</td><td>109%</t

In [196]:
months = []
for year in range(2007,2022):
    year = str(year)
    for month in range(1, 13):
        month = str(month)
        if len(month) == 1:
            months.append(year+'-0'+month)
        elif len(month) == 2:
            months.append(year+'-'+month)
months = months[:-6]

In [197]:
%%time 
monthly_rainfall = []
for i in months: 
    url = f"https://cdec.water.ca.gov/reportapp/javareports?name=PRECIPSUM.{i.replace('-', '')}"
    page = requests.get(url)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    rain = soup.find_all(id='PRECIPSUM')
    
    region = rain[1]
    region = region.find_all('tr')[3:13]
    for j in range(len(region)):
        reg = region[j].find_all('td')
        monthly_rainfall.append([
            reg[0].text, i, reg[5].text, reg[6].text, reg[8].text, reg[10].text, 
            ])


CPU times: user 15.2 s, sys: 364 ms, total: 15.6 s
Wall time: 59.1 s


In [198]:
monthly_rainfall

[['NORTH COAST', '2007-01', '18', '17', '23.2%', '44%'],
 ['SAN FRANCISCO BAY', '2007-01', '6', '6', '12.8%', '35%'],
 ['CENTRAL COAST', '2007-01', '10', '8', '29.5%', '27%'],
 ['SOUTH COAST', '2007-01', '12', '12', '23.9%', '13%'],
 ['SACRAMENTO RIVER', '2007-01', '37', '37', '10.8%', '30%'],
 ['SAN JOAQUIN RIVER', '2007-01', '24', '24', '24.3%', '27%'],
 ['TULARE LAKE', '2007-01', '29', '28', '22.1%', '20%'],
 ['NORTH LAHONTAN', '2007-01', '13', '12', '23.3%', '21%'],
 ['SOUTH LAHONTAN', '2007-01', '15', '10', '15.6%', '15%'],
 ['COLORADO RIVER', '2007-01', '6', '6', '7.5%', '4%'],
 ['NORTH COAST', '2007-02', '18', '18', '151.4%', '66%'],
 ['SAN FRANCISCO BAY', '2007-02', '6', '6', '144.7%', '60%'],
 ['CENTRAL COAST', '2007-02', '10', '8', '93.7%', '45%'],
 ['SOUTH COAST', '2007-02', '12', '12', '46.2%', '22%'],
 ['SACRAMENTO RIVER', '2007-02', '37', '36', '150.5%', '54%'],
 ['SAN JOAQUIN RIVER', '2007-02', '24', '24', '137.4%', '50%'],
 ['TULARE LAKE', '2007-02', '29', '29', '99.4%'

In [199]:
# columns = ['hydrologic_region', 'date', 'stations', 'pct_hist_avg', 'pct_hist_avg_year']
# monthly_rainfall = pd.DataFrame(monthly_rainfall, columns=columns)

# columns = ['hydrologic_region', 'date', 'stations', 'pct_hist_avg', 'pct_hist_avg_year']
monthly_rainfall = pd.DataFrame(monthly_rainfall)
monthly_rainfall.columns = ['hydrologic_region', 'date', 'stations', 'precip', 'pct_hist_avg', 'pct_hist_avg_year']
for i in ['pct_hist_avg', 'pct_hist_avg_year']:
    monthly_rainfall[i] = monthly_rainfall[i].str.strip('%').astype(float)
# monthly_rainfall['date'] = pd.to_datetime(monthly_rainfall['date'].astype(str))
monthly_rainfall.dtypes

hydrologic_region     object
date                  object
stations              object
precip                object
pct_hist_avg         float64
pct_hist_avg_year    float64
dtype: object

In [200]:
monthly_rainfall['date'] = pd.to_datetime(monthly_rainfall['date'].astype(str))

In [201]:
monthly_rainfall.dtypes

hydrologic_region            object
date                 datetime64[ns]
stations                     object
precip                       object
pct_hist_avg                float64
pct_hist_avg_year           float64
dtype: object

In [202]:
monthly_rainfall.columns

Index(['hydrologic_region', 'date', 'stations', 'precip', 'pct_hist_avg',
       'pct_hist_avg_year'],
      dtype='object')

---
## Export
---

In [203]:
monthly_rainfall.to_csv('./cleaned_datasets/rainfall.csv', index=False)